# Masked Likelihoods

In [ ]:
import sys, os
from pyprojroot import here

# spyder up to find the root
root = here(project_files=[".local"])

# append to path
sys.path.append(str(root))

%load_ext autoreload
%autoreload 2

In [ ]:
import torch
import torch.nn as nn
import torch.distributions as dist
import numpy as np
from einops import repeat, rearrange

In [ ]:
from torchkf.kf import transition_predict, emission_predict, update_step, predict_step
from torchkf.kf import (
    mask_observation_noise_diag,
    masked_multivariate_likelihood,
    mask_observation_operator,
)

### Data

$$
y = [u, v]
$$

$$
Y = [y_1, y_2]
$$

$$
Y = [u_1, v_1, u_2, v_2]
$$

$$
Y = [u_1, v_1, 0, 0]
$$

### Observation Mask


$$
\mathbf{H}_t = 
\begin{bmatrix}
\mathbf{H}_t^{\text{obs}} \\
\mathbf{H}_t^{\text{missing}}
\end{bmatrix} =
\begin{bmatrix}
\mathbf{H}_t^{\text{obs}} \\
\mathbf{0}
\end{bmatrix}
$$

In [ ]:
mask_observation_operator??

In [ ]:
state_dim = 5
obs_dim = 2

# operator = np.ones((obs_dim, state_dim))
operator = np.random.randn(obs_dim, state_dim)
operator = torch.Tensor(operator)
# no mask
mask = np.zeros(obs_dim)
mask = torch.Tensor(mask)
print(mask)

In [ ]:
operator_masked = mask_observation_operator(operator, mask)

assert operator_masked.shape == operator.shape
# torch.testing.assert_equal(operator_masked, operator)

print(operator_masked)

Nothing was masked! because we see all of the values :).

In [ ]:
# no mask
mask = torch.Tensor([1, 0])

print(mask)

In [ ]:
operator_masked = mask_observation_operator(operator, mask)

assert operator_masked.shape == operator.shape

print(operator_masked)

As we can see, now we have the operator masked for one of the dimensions.

### Noise Covariance Matrix

mask_observation_operator??$$
\mathbf{R}_t = 
\begin{bmatrix}
\mathbf{R}_{11t}^{\text{obs}} & \mathbf{R}_{12t}^{\text{cross}}\\
\mathbf{R}_{21t}^{\text{cross}} & \mathbf{R}_{22t}^{\text{missing}}
\end{bmatrix} =
\begin{bmatrix}
\mathbf{R}_{11t}^{\text{obs}} & \mathbf{0}\\
\mathbf{0} & \mathbf{I}
\end{bmatrix}
$$

In [ ]:
mask_observation_noise??

In [ ]:
state_dim = 10
obs_dim = 4
rng = np.random.RandomState(3)

# noise_mat = np.ones((obs_dim, obs_dim))
noise_mat = np.random.randn(obs_dim, obs_dim)
print(noise_mat)
noise_mat = torch.Tensor(noise_mat)

In [ ]:
mask = np.zeros(obs_dim)

print(mask)

mask = torch.Tensor(mask)

In [ ]:
noise_mat_masked = mask_observation_noise_diag(noise_mat, mask)

assert noise_mat_masked.shape == noise_mat.shape
torch.testing.assert_equal(noise_mat_masked, noise_mat)

print(noise_mat_masked)

Again, we don't see any changes because the mask was 1s everywhere.

In [ ]:
mask = np.array([0.0, 0.0, 1.0, 1.0])
mask = torch.Tensor(mask)

noise_mat_masked = mask_observation_noise_diag(noise_mat, mask)

assert noise_mat_masked.shape == noise_mat.shape

print(noise_mat_masked)

here, we see the structure we showed above.

In [ ]:
mask = np.array([1.0, 0.0, 0.0, 1.0])
mask = torch.tensor(mask)

noise_mat_masked = mask_observation_noise_diag(noise_mat, mask)


assert noise_mat_masked.shape == noise_mat_masked.shape

print(noise_mat_masked)

### Likelihood Term

In [ ]:
obs_dim = 5

rng = np.random.RandomState(123)

x = rng.randn(obs_dim)
mean = rng.randn(obs_dim)
cov = rng.randn(obs_dim, obs_dim)
cov = cov @ cov.T

In [ ]:
log_prob = masked_multivariate_likelihood(
    torch.Tensor(x), torch.Tensor(mean), torch.Tensor(cov), None
)
assert log_prob.shape == ()
assert np.isnan(log_prob).sum() == 0
log_prob

In [ ]:
log_prob = masked_multivariate_likelihood(
    torch.Tensor(x),
    torch.Tensor(mean),
    torch.Tensor(cov),
    torch.zeros_like(torch.Tensor(x)),
)
assert log_prob.shape == ()
assert np.isnan(log_prob).sum() == 0
log_prob

In [ ]:
log_prob = masked_multivariate_likelihood(
    torch.Tensor(x),
    torch.Tensor(mean),
    torch.Tensor(cov),
    torch.ones_like(torch.Tensor(x)),
)
assert log_prob.shape == ()
assert np.isnan(log_prob).sum() == 0
log_prob

In [ ]:
mask = rng.randint(
    0,
    2,
    size=(obs_dim,),
).astype(np.float32)

In [ ]:
mask

In [ ]:
log_prob = masked_multivariate_likelihood(
    torch.Tensor(x), torch.Tensor(mean), torch.Tensor(cov), torch.Tensor(mask)
)
assert log_prob.shape == ()
assert np.isnan(log_prob).sum() == 0
log_prob

##### Batches

In [ ]:
obs_dim = 2
n_batch = 10
x = rng.randn(n_batch, obs_dim)
mean = rng.randn(n_batch, obs_dim)
cov = rng.randn(n_batch, obs_dim, obs_dim)
cov = jnp.matmul(cov, cov.transpose((0, 2, 1)))

In [ ]:
fn = jax.vmap(mvn_logpdf, in_axes=(0, 0, 0, None))
# fn_ = jax.vmap(mvn_logpdf_custom, in_axes=(0, 0, 0, None))

log_probs = fn(x, mean, cov, None)
# log_probs_ = fn_(x, mean, cov, None)
# chex.assert_trees_all_close(log_probs, log_probs_)
assert log_probs.shape == (n_batch,)
assert np.isnan(log_probs).sum() == 0

In [ ]:
log_probs